In [ ]:
import pandas as pd
import numpy as np

#Customers
customers = pd.DataFrame({
    "cust_id": [1, 2, 3, 4],
    "name": ["Alice", "Bob", "Carol", "Dave"],
    "type": ["Individual", "Individual", "Business", "Business"]
})

#Accounts (Many per customer)
accounts = pd.DataFrame({
    "acc_id": [101, 102, 103, 104, 105],
    "cust_id": [1, 2, 2, 3, 4],
    "acc_type": ["Savings", "Savings", "Current", "Current", "Savings"],
    "opened": pd.to_datetime([
        "2022-01-01", "2022-05-20", "2023-02-12", "2022-08-01", "2023-01-11"
    ])
})

#Transactions (multiple per account)
transactions = pd.DataFrame({
    "txn_id": range(1,11),
    "acc_id": [101,101,102,102,103,104,104,105,105,105],
    "date": pd.to_datetime([
        "2024-01-01", "2024-02-10", "2024-01-20", "2024-03-05", "2024-02-28", 
        "2024-01-15", "2024-03-08", "2024-01-18", "2024-02-14", "2024-03-12"
    ]),
    "t_type": ["credit", "debit", "credit", "debit", "credit", 
             "debit", "credit", "credit", "debit", "credit"],  # positive/negative cash flow
    "amount": [10000, 3000, 15000, 5000, 45000, 
               7000, 25000, 12000, 4000, 8000]
})


In [2]:
# Merge account, customer info to transactions
txn_full = transactions.merge(accounts, on="acc_id").merge(customers, on="cust_id")
print(txn_full)


   txn_id  acc_id       date  type_x  amount  cust_id acc_type     opened  \
0       1     101 2024-01-01  credit   10000        1  Savings 2022-01-01   
1       2     101 2024-02-10   debit    3000        1  Savings 2022-01-01   
2       3     102 2024-01-20  credit   15000        2  Savings 2022-05-20   
3       4     102 2024-03-05   debit    5000        2  Savings 2022-05-20   
4       5     103 2024-02-28  credit   45000        2  Current 2023-02-12   
5       6     104 2024-01-15   debit    7000        3  Current 2022-08-01   
6       7     104 2024-03-08  credit   25000        3  Current 2022-08-01   
7       8     105 2024-01-18  credit   12000        4  Savings 2023-01-11   
8       9     105 2024-02-14   debit    4000        4  Savings 2023-01-11   
9      10     105 2024-03-12  credit    8000        4  Savings 2023-01-11   

    name      type_y  
0  Alice  Individual  
1  Alice  Individual  
2    Bob  Individual  
3    Bob  Individual  
4    Bob  Individual  
5  Carol    Bu

In [8]:
# Add signed "amount" column using the correct transaction type column
txn_full["signed_amt"] = np.where(txn_full["type_x"]=="credit", txn_full["amount"], -txn_full["amount"])
print(txn_full[["txn_id","acc_id","amount","type_x","signed_amt"]])


   txn_id  acc_id  amount  type_x  signed_amt
0       1     101   10000  credit       10000
1       2     101    3000   debit       -3000
2       3     102   15000  credit       15000
3       4     102    5000   debit       -5000
4       5     103   45000  credit       45000
5       6     104    7000   debit       -7000
6       7     104   25000  credit       25000
7       8     105   12000  credit       12000
8       9     105    4000   debit       -4000
9      10     105    8000  credit        8000


In [9]:
# Compute running balance per account
txn_full = txn_full.sort_values(["acc_id","date","txn_id"])
txn_full["balance"] = (
    txn_full.groupby("acc_id")["signed_amt"].cumsum()
)
print(txn_full[["acc_id","date","signed_amt","balance"]])


   acc_id       date  signed_amt  balance
0     101 2024-01-01       10000    10000
1     101 2024-02-10       -3000     7000
2     102 2024-01-20       15000    15000
3     102 2024-03-05       -5000    10000
4     103 2024-02-28       45000    45000
5     104 2024-01-15       -7000    -7000
6     104 2024-03-08       25000    18000
7     105 2024-01-18       12000    12000
8     105 2024-02-14       -4000     8000
9     105 2024-03-12        8000    16000


In [11]:
# Total deposits/withdrawals per account and customer
totals_acc = txn_full.pivot_table(index="acc_id", columns="type_x", values="amount", aggfunc="sum", fill_value=0)
totals_cust = txn_full.groupby(["cust_id","type_x"])["amount"].sum().unstack(fill_value=0)
print(totals_acc)
print(totals_cust)


type_x  credit  debit
acc_id               
101      10000   3000
102      15000   5000
103      45000      0
104      25000   7000
105      20000   4000
type_x   credit  debit
cust_id               
1         10000   3000
2         60000   5000
3         25000   7000
4         20000   4000


In [12]:
# End-of-month ("EOM") account balance over time 
# Set date as index for resampling
ts = txn_full.set_index("date")
eom_bal = (
    ts.groupby("acc_id")["balance"]
      .resample("M")
      .last()
      .unstack(level=0)
)
print(eom_bal)


acc_id          101      102      103      104      105
date                                                   
2024-01-31  10000.0  15000.0      NaN  -7000.0  12000.0
2024-02-29   7000.0      NaN  45000.0      NaN   8000.0
2024-03-31      NaN  10000.0      NaN  18000.0  16000.0


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_120\3434249321.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")


In [13]:
# Customer summaries: total balance, recent activity
summary = (
    txn_full.groupby("cust_id").agg(
        total_credits = ("signed_amt", lambda x: x[x>0].sum()),
        total_debits = ("signed_amt", lambda x: -x[x<0].sum()),
        last_txn = ("date","max"),
        latest_balance = ("balance","last")
    )
)
print(summary.join(customers.set_index("cust_id")))


         total_credits  total_debits   last_txn  latest_balance   name  \
cust_id                                                                  
1                10000          3000 2024-02-10            7000  Alice   
2                60000          5000 2024-03-05           45000    Bob   
3                25000          7000 2024-03-08           18000  Carol   
4                20000          4000 2024-03-12           16000   Dave   

               type  
cust_id              
1        Individual  
2        Individual  
3          Business  
4          Business  


In [14]:
# Rolling 3-transaction moving average for account balances
txn_full["roll_avg_bal"] = (
    txn_full.groupby("acc_id")["balance"]
            .rolling(3,min_periods=1).mean().reset_index(level=0,drop=True)
)
print(txn_full[["acc_id","date","balance","roll_avg_bal"]])


   acc_id       date  balance  roll_avg_bal
0     101 2024-01-01    10000       10000.0
1     101 2024-02-10     7000        8500.0
2     102 2024-01-20    15000       15000.0
3     102 2024-03-05    10000       12500.0
4     103 2024-02-28    45000       45000.0
5     104 2024-01-15    -7000       -7000.0
6     104 2024-03-08    18000        5500.0
7     105 2024-01-18    12000       12000.0
8     105 2024-02-14     8000       10000.0
9     105 2024-03-12    16000       12000.0
